# Topic Router

This notebook assigns topics to reviews using LLM-powered multi-label classification.


In [1]:
import sys
sys.path.append('../')

import polars as pl
import json
import os
from pathlib import Path
from datetime import datetime, timedelta, timezone
from utils.llm_client import LLMClient
os.environ['MEGALLM_API_KEY'] = 'sk-mega-5c718f7e9327ca90c8dbf159e39a4192407c48474a922ec5fa6a91027a247d1a'
os.environ['MEGALLM_BASE_URL'] = 'https://ai.megallm.io/v1'
os.environ.setdefault('MEGALLM_MODEL', 'gpt-4o-mini')

from tqdm import tqdm
import hashlib

# Set up paths
DATA_DIR = Path("../data")
REVIEWS_FILE = DATA_DIR / "reviews_clean.parquet"
REGISTRY_FILE = Path("../registry/topic_registry.json")
OUTPUT_FILE = DATA_DIR / "labels_initial.parquet"

IST_TZ = timezone(timedelta(hours=5, minutes=30))
START_DATE = datetime(2024, 6, 1, tzinfo=IST_TZ)
TARGET_DATE = datetime.now(IST_TZ).date()
ROLLING_WINDOW_DAYS = 30
DAILY_REVIEWS_DIR = DATA_DIR / "daily_batches"
DAILY_LABELS_DIR = DATA_DIR / "daily_labels"

for path in [DATA_DIR, DAILY_REVIEWS_DIR, DAILY_LABELS_DIR]:
    path.mkdir(exist_ok=True)


print("✓ Setup complete")

os.environ.setdefault('ROUTING_MAX_WORKERS', '50')

✓ Setup complete


'50'

## Load Data and Registry


In [2]:
# Load reviews
reviews_df = pl.read_parquet(REVIEWS_FILE)
if 'created_at' not in reviews_df.columns:
    raise ValueError('Expected created_at column in reviews parquet.')
print(f"✓ Loaded {len(reviews_df):,} reviews")

reviews_df = reviews_df.with_columns([
    pl.col('created_at').dt.convert_time_zone('Asia/Kolkata').alias('created_at'),
    pl.col('created_at').dt.date().alias('dt')
])


✓ Loaded 225,918 reviews


## Initialize LLM Client


In [3]:
PROVIDER = 'megallm'
MODEL = os.getenv('MEGALLM_MODEL', os.getenv('ROUTING_MODEL', 'gpt-4o-mini'))

In [4]:
llm = LLMClient(provider=PROVIDER, model=MODEL)
print(f'✓ Initialized LLM client: {PROVIDER} with model {MODEL}')

✓ Initialized megallm client with model gpt-4o-mini
✓ Initialized LLM client: megallm with model gpt-4o-mini


## Load Topic Registry

In [5]:
# Load topic registry
with open(REGISTRY_FILE) as f:
    registry = json.load(f)

registry_topics = registry.get("topics", [])
if not registry_topics:
    raise ValueError(f"No topics loaded from registry: {REGISTRY_FILE}")

topic_lookup = {topic["id"]: topic for topic in registry_topics}
print(f"✓ Loaded {len(registry_topics)} topics from registry")

✓ Loaded 32 topics from registry


## Routing Helpers

## Create Router Function


## Batch Process Reviews

Process all reviews in batches with caching


In [6]:
from textwrap import dedent
from typing import Any, Dict, List
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock
import time
import hashlib

ROUTING_SYSTEM_PROMPT = (
    "You are a high-recall topic routing assistant for Swiggy reviews. "
    "Use the provided topic catalog to assign every relevant topic id. "
    "Return JSON with keys topic_ids (list[str]), is_novel (bool), novel (object or null). "
    "Prefer recall but avoid assigning unrelated topics."
)

MAX_TOPICS_PER_REVIEW = 4
ROUTING_TEMPERATURE = 0.2
MODEL_COST_USD_PER_1K = {
    'mega-1-chat': {'input': 0.0005, 'output': 0.0015},
    'gpt-4o-mini': {'input': 0.0006, 'output': 0.0024},
    'gpt-4o': {'input': 0.0025, 'output': 0.01},
    'claude-3-haiku-20240307': {'input': 0.00025, 'output': 0.00125},
    'claude-3-5-sonnet-20240620': {'input': 0.003, 'output': 0.015},
}
DEFAULT_COST = {'input': 0.0010, 'output': 0.0020}
MAX_COST_USD = float(os.getenv('ROUTING_COST_BUDGET_USD', '25'))
OUTPUT_TOKEN_ESTIMATE = 180  # heuristic budget per call
PROMPT_BACKOFF = 1.5
ROUTING_MAX_WORKERS = int(os.getenv('ROUTING_MAX_WORKERS', '4'))

def build_topic_catalog(topics):
    lines = []
    for topic in topics:
        examples = topic.get('positive_examples') or []
        example_snippet = '; '.join(
            example.strip()
            for example in examples[:2]
            if isinstance(example, str) and example.strip()
        )
        definition = (topic.get('definition') or '').strip()
        if example_snippet:
            lines.append(
                f"{topic['id']} :: {topic['name']} — {definition} (e.g. {example_snippet})"
            )
        else:
            lines.append(f"{topic['id']} :: {topic['name']} — {definition}")
    return '\n'.join(lines)

def estimate_tokens(text):
    if not text:
        return 0
    return max(1, len(text) // 4)

TOPIC_CATALOG = build_topic_catalog(registry_topics)
PROMPT_OVERHEAD_TOKENS = estimate_tokens(TOPIC_CATALOG) + 150

def estimate_cost(input_tokens, output_tokens, model):
    rates = MODEL_COST_USD_PER_1K.get(model, DEFAULT_COST)
    return (input_tokens * rates['input'] + output_tokens * rates['output']) / 1000.0

def build_user_prompt(review_text):
    review_text = (review_text or '').strip()
    return dedent(f"""\
        Review text:
        "{review_text}"

        Topic catalog:
        {TOPIC_CATALOG}

        Instructions:
        - Reply strictly in JSON with keys:
          - topic_ids: up to {MAX_TOPICS_PER_REVIEW} topic IDs from the catalog that apply.
          - is_novel: true if the review exposes a new issue not covered in the catalog.
          - novel: when is_novel is true, include an object with keys label (<=5 words) and rationale.
        - Capture every relevant topic even for positive sentiment.
        - Prefer existing topics when the description is close to a catalog entry.
        - Return an empty array for topic_ids when nothing applies.
        """)

def validate_topic_ids(topic_ids):
    cleaned = []
    if isinstance(topic_ids, str):
        topic_ids = [topic_ids]
    if not isinstance(topic_ids, list):
        return cleaned
    for topic_id in topic_ids:
        if not isinstance(topic_id, str):
            continue
        normalized = topic_id.strip().upper()
        if normalized in topic_lookup and normalized not in cleaned:
            cleaned.append(normalized)
    return cleaned

def route_review(
    review_text,
    llm_client,
    max_retries=3,
    retry_backoff=PROMPT_BACKOFF,
):
    if not review_text or not review_text.strip():
        return {
            'topic_ids': [],
            'is_novel': False,
            'novel': None,
            'input_tokens_est': 0,
            'output_tokens_est': OUTPUT_TOKEN_ESTIMATE,
            'routing_error': 'empty_review',
        }

    input_estimate = estimate_tokens(review_text) + PROMPT_OVERHEAD_TOKENS
    attempt = 0
    last_error = ''
    while attempt < max_retries:
        try:
            response = llm_client.complete(
                system_prompt=ROUTING_SYSTEM_PROMPT,
                user_prompt=build_user_prompt(review_text),
                temperature=ROUTING_TEMPERATURE,
                response_format='json',
                use_cache=True,
            ) or {}
            topic_ids = response.get('topic_ids', []) or response.get('topics', [])
            cleaned_topics = validate_topic_ids(topic_ids)
            novel_payload = response.get('novel') if isinstance(response.get('novel'), dict) else None
            is_novel = bool(response.get('is_novel') and novel_payload)
            return {
                'topic_ids': cleaned_topics,
                'is_novel': is_novel,
                'novel': novel_payload,
                'input_tokens_est': input_estimate,
                'output_tokens_est': OUTPUT_TOKEN_ESTIMATE,
                'routing_error': None,
            }
        except Exception as exc:
            attempt += 1
            last_error = repr(exc)
            wait_seconds = min(8.0, retry_backoff ** attempt)
            print(f"  ⚠️ Routing failed (attempt {attempt}/{max_retries}): {exc}")
            time.sleep(wait_seconds)

    return {
        'topic_ids': [],
        'is_novel': False,
        'novel': None,
        'input_tokens_est': input_estimate,
        'output_tokens_est': OUTPUT_TOKEN_ESTIMATE,
        'routing_error': last_error or 'unknown_error',
    }


In [7]:
def route_batch(rows: List[Dict[str, Any]], llm_client: LLMClient, batch_desc: str, cache: Dict[str, Dict[str, Any]]) -> List[Dict[str, Any]]:
    if not rows:
        return []

    max_workers = max(1, ROUTING_MAX_WORKERS)
    if max_workers == 1 or len(rows) == 1:
        results: List[Dict[str, Any]] = []
        for row in tqdm(rows, desc=f"Routing {batch_desc}", leave=False):
            review_text = row['content_raw']
            text_hash = hashlib.sha256(review_text.encode('utf-8')).hexdigest()
            if text_hash in cache:
                cached_result = cache[text_hash]
                routed = cached_result.copy()
                routed['from_cache'] = True
            else:
                routed = route_review(review_text, llm_client)
                cache[text_hash] = routed.copy()
                routed['from_cache'] = False
            results.append({**row, **routed})
        return results

    results: List[Dict[str, Any]] = [None] * len(rows)
    cache_lock = Lock()

    def process_item(idx_row):
        idx, row = idx_row
        review_text = row['content_raw']
        text_hash = hashlib.sha256(review_text.encode('utf-8')).hexdigest()
        with cache_lock:
            cached_result = cache.get(text_hash)
        if cached_result is not None:
            routed = cached_result.copy()
            routed['from_cache'] = True
        else:
            routed = route_review(review_text, llm_client)
            with cache_lock:
                cache[text_hash] = routed.copy()
            routed['from_cache'] = False
        return idx, {**row, **routed}

    indexed_rows = list(enumerate(rows))
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_item, item) for item in indexed_rows]
        for future in tqdm(as_completed(futures), total=len(futures), desc=f"Routing {batch_desc}", leave=False):
            idx, routed_row = future.result()
            results[idx] = routed_row
    return results

In [8]:
# Configure daily processing window
target_date = TARGET_DATE
if target_date < START_DATE.date():
    raise ValueError('Target date precedes June 1 2024. Adjust TARGET_DATE.')
window_start = target_date - timedelta(days=ROLLING_WINDOW_DAYS)
print(f'Processing window: {window_start} to {target_date}')

# Ensure date column exists
if 'dt' not in reviews_df.columns:
    reviews_df = reviews_df.with_columns([
        pl.col('created_at').dt.convert_time_zone('Asia/Kolkata').dt.date().alias('dt')
    ])

unique_dates = sorted(set(reviews_df['dt'].to_list()))
if not unique_dates:
    raise ValueError('No review dates available for routing.')
print(f'Total candidate days: {len(unique_dates)}')

all_labels = []
cache: Dict[str, Dict[str, Any]] = {}
total_review_count = 0
total_assignments = 0
total_novel = 0
total_input_tokens = 0
total_output_tokens = 0
total_cost_est = 0.0
routing_errors = 0

for batch_date in unique_dates:
    if batch_date < window_start or batch_date > target_date:
        continue
    day_reviews = reviews_df.filter(pl.col('dt') == batch_date).sort('created_at')
    day_key = batch_date.isoformat()
    print(f'📅 Processing {day_key}: {len(day_reviews)} reviews')

    if len(day_reviews) == 0:
        print('  → No reviews for this day.')
        continue

    day_reviews_path = DAILY_REVIEWS_DIR / f'reviews_{day_key}.parquet'
    day_reviews.write_parquet(day_reviews_path)

    day_rows = [
        {
            'review_id': row['review_id'],
            'content_raw': row['content_raw'],
            'created_at': row['created_at'],
            'dt': row['dt'],
        }
        for row in day_reviews.iter_rows(named=True)
    ]

    batch_results = route_batch(day_rows, llm, batch_desc=day_key, cache=cache)

    day_labels = []
    day_assignments = 0
    day_novel = 0
    day_errors = 0
    day_cost_est = 0.0

    for routed in batch_results:
        total_review_count += 1

        input_tokens = routed.get('input_tokens_est', 0)
        output_tokens = routed.get('output_tokens_est', OUTPUT_TOKEN_ESTIMATE)
        from_cache = routed.get('from_cache', False)
        call_cost = 0.0
        if not from_cache:
            call_cost = estimate_cost(input_tokens, output_tokens, MODEL)
            total_input_tokens += input_tokens
            total_output_tokens += output_tokens
            total_cost_est += call_cost
            day_cost_est += call_cost

            if total_cost_est > MAX_COST_USD:
                raise RuntimeError(
                    f"Estimated routing cost ${total_cost_est:.2f} exceeds budget ${MAX_COST_USD:.2f}. "
                    'Set ROUTING_COST_BUDGET_USD to raise the limit or reduce the date range.'
                )

        topic_ids = routed.get('topic_ids', [])
        if topic_ids:
            for topic_id in topic_ids:
                day_labels.append({
                    'review_id': routed['review_id'],
                    'topic_id': topic_id,
                    'is_novel': False,
                    'novel_label': None,
                    'novel_rationale': None,
                    'created_at': routed['created_at'],
                    'dt': routed['dt'],
                })
                day_assignments += 1
                total_assignments += 1

        if routed.get('is_novel') and isinstance(routed.get('novel'), dict):
            novel = routed['novel']
            day_labels.append({
                'review_id': routed['review_id'],
                'topic_id': 'NOVEL',
                'is_novel': True,
                'novel_label': novel.get('label'),
                'novel_rationale': novel.get('rationale'),
                'created_at': routed['created_at'],
                'dt': routed['dt'],
            })
            day_novel += 1
            total_novel += 1

        if routed.get('routing_error'):
            routing_errors += 1
            day_errors += 1

    print(f'  → Routed {len(batch_results)} reviews | assignments: {day_assignments} | novel: {day_novel} | errors: {day_errors} | est cost: ${day_cost_est:.2f}')

    if day_labels:
        day_labels_df = pl.DataFrame(day_labels)
        day_labels_path = DAILY_LABELS_DIR / f'labels_{day_key}.parquet'
        day_labels_df.write_parquet(day_labels_path)
        print(f'    Saved {len(day_labels)} label rows to {day_labels_path.name}')
        all_labels.extend(day_labels)
    else:
        print('    No topics detected for this day.')

print('=== Routing Summary ===')
print(f'Total reviews routed: {total_review_count}')
print(f'Total topic assignments: {total_assignments}')
print(f'Novel reviews flagged: {total_novel}')
print(f'Estimated tokens (input/output): {total_input_tokens}/{total_output_tokens}')
print(f'Estimated cost: ${total_cost_est:.2f} (budget ${MAX_COST_USD:.2f})')
if total_cost_est > MAX_COST_USD * 0.9:
    print('⚠️ Estimated cost is approaching the configured budget.')
if routing_errors:
    print(f'⚠️ {routing_errors} reviews encountered routing errors. Consider rerunning with higher retry count.')
else:
    print('✓ No routing errors detected.')

Processing window: 2025-09-28 to 2025-10-28
Total candidate days: 513
📅 Processing 2025-09-28: 776 reviews


  → Routed 776 reviews | assignments: 1135 | novel: 107 | errors: 0 | est cost: $0.64
    Saved 1242 label rows to labels_2025-09-28.parquet
📅 Processing 2025-09-29: 489 reviews


  → Routed 489 reviews | assignments: 770 | novel: 70 | errors: 0 | est cost: $0.40
    Saved 840 label rows to labels_2025-09-29.parquet
📅 Processing 2025-09-30: 571 reviews


  → Routed 571 reviews | assignments: 839 | novel: 83 | errors: 0 | est cost: $0.45
    Saved 922 label rows to labels_2025-09-30.parquet
📅 Processing 2025-10-01: 554 reviews


  → Routed 554 reviews | assignments: 841 | novel: 71 | errors: 0 | est cost: $0.43
    Saved 912 label rows to labels_2025-10-01.parquet
📅 Processing 2025-10-02: 715 reviews


...

  → Routed 500 reviews | assignments: 779 | novel: 77 | errors: 0 | est cost: $0.37
    Saved 856 label rows to labels_2025-10-24.parquet
📅 Processing 2025-10-25: 499 reviews


  → Routed 499 reviews | assignments: 733 | novel: 87 | errors: 0 | est cost: $0.36
    Saved 820 label rows to labels_2025-10-25.parquet
📅 Processing 2025-10-26: 305 reviews


Routing 2025-10-26:  78%|████████████████▍    | 239/305 [00:08<00:02, 23.72it/s]

  "topic_ids": [
    "POSITIVE_EXPERIENCE",
    "FAST_DELIVERY"
  ],



RuntimeError: Estimated routing cost $12.00 exceeds budget $12.00. Set ROUTING_COST_BUDGET_USD to raise the limit or reduce the date range.

## Save Results and Show Distribution


In [9]:
if not all_labels:
    raise ValueError('No routing labels generated. Run the routing step before saving results.')

labels_df = pl.DataFrame(all_labels)

# Save to Parquet
labels_df.write_parquet(OUTPUT_FILE)
print(f"✓ Saved labels to {OUTPUT_FILE} ({len(labels_df)} rows)")

# Show distribution
print("📊 Topic Distribution:")
print(labels_df.group_by('topic_id').agg(pl.len().alias('count')).sort('count', descending=True))

✓ Saved labels to ../data/labels_initial.parquet (27381 rows)
📊 Topic Distribution:
shape: (33, 2)
┌─────────────────────┬───────┐
│ topic_id            ┆ count │
│ ---                 ┆ ---   │
│ str                 ┆ u32   │
╞═════════════════════╪═══════╡
│ POSITIVE_EXPERIENCE ┆ 8092  │
│ NEGATIVE_GENERIC    ┆ 2781  │
│ NOVEL               ┆ 2401  │
│ VERY_GOOD_SERVICE   ┆ 1845  │
│ LATE_DELIVERY       ┆ 1475  │
│ …                   ┆ …     │
│ LIMITED_OPTIONS     ┆ 50    │
│ OUT_OF_STOCK        ┆ 37    │
│ PACKAGING_LEAK      ┆ 26    │
│ CART_BUG            ┆ 18    │
│ OTP_ISSUE           ┆ 7     │
└─────────────────────┴───────┘
